# Exercises 24 February 2021

## Personal networks, ego networks

In [ ]:
########### Preparation ##############
# import packages
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# use pandas to import list of nodes with attributes
nodes = pd.read_table('Ego210_Nodes.txt', sep='\t')
print(nodes)

In [ ]:
# Let's select Ego
ego = nodes[nodes.ID==210]
print(ego)

In [ ]:
# Select alters 
alters = nodes[nodes.ID != 210]
print(alters)

In [ ]:
# use pandas to import edgelist as a table
EgoEdges = pd.read_table('Ego210_Edges.txt', sep=',')
EgoEdges

In [ ]:
# generate graph from pandas edgelist
# it is an undirected multigraph
# a 'multigraph' accepts multiple types of ties (here, online and offline)

Ego210 = nx.from_pandas_edgelist(EgoEdges, 'V1', 'V2', edge_attr ='Type', create_using=nx.MultiGraph()) 
print(nx.info(Ego210))

In [ ]:
# Note: if we had created it as a simple undirected graph, networkx would have omitted multiple ties

test = nx.from_pandas_edgelist(EgoEdges, 'V1', 'V2', create_using=nx.Graph()) 
print(nx.info(test))

In [ ]:
# match nodes of graph with attributes of nodes
NodeData = nodes.set_index('ID').to_dict('index').items()
Ego210.add_nodes_from(NodeData)

# view results
print(Ego210.nodes(data=True))

In [ ]:
# view edges
print(Ego210.edges(data=True))

In [ ]:
# Visualization

# define layout
pos = nx.spring_layout(Ego210)
# draw nodes according to layoabsut
nx.draw_networkx_nodes(Ego210, pos, node_color = 'r', node_size = 100, alpha = 1)
# use matplotlib to draw edges
ax = plt.gca()
for e in Ego210.edges:
    ax.annotate("",
                xy=pos[e[0]], xycoords='data',
                xytext=pos[e[1]], textcoords='data',
                arrowprops=dict(arrowstyle="-",
                                color='gray',
                                shrinkA=5, shrinkB=5,
                                patchA=None, patchB=None,
                                connectionstyle="arc3,rad=rrr".replace('rrr',str(0.3*e[2])),
                                ),
                )
plt.axis('off')
plt.show()

## Subset graph: offline-only, online-only, without Ego

In [ ]:
## Subset graph: take only offline ties

# take subset of relevant edges
offline = [(u, v) for (u, v, d) in Ego210.edges(data=True) if d["Type"] ==1]

# create graph
Ego210Offline = nx.Graph(offline)
print(nx.info(Ego210Offline))

In [ ]:
# match with nodes table to add isolates
Ego210Offline.add_nodes_from(NodeData)

# view results
print(nx.info(Ego210Offline))

In [ ]:
# Visualization

# keep the same layout as before ("pos") to compare
# draw nodes according to this layout
nx.draw_networkx_nodes(Ego210Offline, pos, node_color = 'r', node_size = 100, alpha = 1)
# use matplotlib to draw edges
ax = plt.gca()
for e in Ego210Offline.edges:
    ax.annotate("",
                xy=pos[e[0]], xycoords='data',
                xytext=pos[e[1]], textcoords='data',
                arrowprops=dict(arrowstyle="-",
                                color='gray',
                                shrinkA=5, shrinkB=5,
                                patchA=None, patchB=None,
                                connectionstyle="arc3,rad=0.3"),
                                )
plt.axis('off')
plt.show()

##### Exercise: now, repeat the same for online ties

In [ ]:
# Graph without Ego
Ego210_noEgo = Ego210.copy()
Ego210_noEgo.remove_node(210)

In [ ]:
# View results
print(nx.info(Ego210_noEgo))

##### Exercise: now, visualize it with the same layout as Ego210

In [ ]:
## Create weighted undirected graph from multigraph
# Idea : each tie has value 2 if it is both online and offline, 1 otherwise
# We can interpret multiplexity as tie strength

Ego210WUG = nx.Graph()

for u,v,data in Ego210.edges(data=True):
    w = 1   
    if Ego210WUG.has_edge(u,v):
        Ego210WUG[u][v]['weight'] += w
    else:
        Ego210WUG.add_edge(u, v, weight=w)

print(nx.info(Ego210WUG))
print(Ego210WUG.edges(data=True))

In [ ]:
# Visualization

strongTies = [(u, v) for (u, v, d) in Ego210WUG.edges(data=True) if d["weight"] > 1]
weakTies = [(u, v) for (u, v, d) in Ego210WUG.edges(data=True) if d["weight"] <= 1]

pos = nx.spring_layout(Ego210WUG)  # positions for all nodes

# nodes
nx.draw_networkx_nodes(Ego210WUG, pos, node_color = 'r', node_size = 100, alpha = 1)

# edges
nx.draw_networkx_edges(Ego210WUG, pos, edgelist=strongTies, width=5)  # strong ties
nx.draw_networkx_edges(Ego210WUG, pos, edgelist=weakTies, width=2) # weak ties


## Personal network composition indicators

In [ ]:
## Blau index

# recall it is equal to 1 - p1^2 - p2^2 - ... - pk^2

# first create a function
def blau(df, col):
    return (1- ((df[col].value_counts() / df[col].count()) ** 2).sum())

# then apply it to the 'alters' table, attribute 'Qualification'
blau(alters, 'Qualification')

In [ ]:
# Index of Qualitative Variation
# equal to Blau * k/(k-1)
# (normalized version)

blau(alters, 'Qualification')*6/5

In [ ]:
# NB if you need to see how many/ which categories are represented in a categorical variables
print (alters['Qualification'].unique())

In [ ]:
# Herfindahl-Hirschman index (HHI)
# equal to 1 - Blau

1 - blau(alters, 'Qualification')

#### Exercise: calculate these indexes for the other attributes (Gender, Multiplexity, Support and Proximity)

## Structural measures

In [ ]:
# find isolates (components consisting in 1 node, for example in the offline-only network)
list(nx.isolates(Ego210Offline)) 

In [ ]:
# density (for ex. in the original Ego210 graph)
nx.density(Ego210) 

In [ ]:
# Transitivity (cannot do it in Ego210 because not implemented for MultiGraph type)
print(nx.transitivity(Ego210WUG)) 

##### In principle, all structural measures can be calculated on a personal / ego network. Just be aware of graph type and relevance of each measure (par ex. reciprocity is not meaningful in un undirected graph).

## Exercises for next week

##### Take the advice network of Lazega's lawyers.

##### Extract the ego-centred network of one of the lawyers i (i = 1,..., 71).
###### NB The networkx function to extract the ego graph of i is: Egoi = nx.ego_graph(G, i)

##### Describe the ego-network of lawyer i through composition indicators (Blau index etc.) for relevant attributes, and structure indicators (density etc.).

##### Bonus: repeat the exercise with the multigraph that results from combining advice, friendship and cowork.

##### Please email to paola.tubaro@lri.fr by Tuesday, March 2, 2pm.
